In [1]:
import os
import time
import datetime
import copy
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import LineString, Point

# 关闭警告提醒
warnings.filterwarnings("ignore")
# 显示所有列
pd.set_option('display.max_columns', None)

# 1. 出行OD统计

*高峰时间段: 7:00-8:00 and 17:00-18:00*
*低峰时间段: 0:00-6:00 and 22:00-23:00*
*平峰时间段: 9:00-16:00 and 19:00-21:00*

In [2]:
# 读取速度估计预处理数据 2021年12月1日 佛山市中心城区燃油客车、货车路径重构数据 对应每一单次出行

folder_path = r'E:\硕士毕业论文update250120\案例分析\数据\路径重构结果\有效处理结果'
test_data = pd.read_csv(folder_path + '/' + 'travel_info.csv', encoding='gbk')
test_data['start_time'] = pd.to_datetime(test_data['start_time'])
test_data['stop_time'] = pd.to_datetime(test_data['stop_time'])

# 按照客车、货车分类
bus_test_data = test_data[test_data['veh_type'].str.contains('客车')]
truck_test_data = test_data[test_data['veh_type'].str.contains('货车')]
lighttruck_test_data = truck_test_data[truck_test_data['veh_type'].isin(['微型货车', '轻型货车'])]
heavytruck_test_data = truck_test_data[truck_test_data['veh_type'].isin(['中型货车', '重型货车'])]

print('2021年12月1日 总出行次数:', len(test_data))
print('客车占比: {:.2%}'.format(len(bus_test_data) / len(test_data)),
      '微轻型货车占比: {:.2%}'.format(len(lighttruck_test_data) / len(test_data)),
      '中重型货车占比: {:.2%}'.format(len(heavytruck_test_data) / len(test_data)))

bus_test_data.head(3)

2021年12月1日 总出行次数: 264422
客车占比: 97.74% 微轻型货车占比: 1.46% 中重型货车占比: 0.80%


,CLTMBH,start_time,stop_time,travel_time_s,predtravel_time_s,distance_km,speed_kmh,bayonet_records,node_records,start_bayonet,stop_bayonet,start_node_lon,start_node_lat,stop_node_lon,stop_node_lat,position,veh_type,fuel_type,pfbz,travel_error
0,粤E00003c875c35df53f1cc6bb188017915,2021-12-01 09:42:43,2021-12-01 10:03:39,1256.0,1644.0,17.77,50.94,4,229,440615000000324015,440604678881002001,113.149607,23.011112,113.063593,23.044394,2,小型客车,汽油,国5,总时间误差满足
1,粤E00003c875c35df53f1cc6bb188017915,2021-12-01 19:19:44,2021-12-01 19:36:09,985.0,1237.0,12.28,44.87,4,217,440604600031007003,440604305050003500,113.139246,23.015031,113.070463,23.047882,3,小型客车,汽油,国5,总时间误差满足
2,粤E000190caca7cf57954e46c53740d2a61,2021-12-01 21:22:15,2021-12-01 21:31:30,555.0,687.0,6.82,44.21,4,64,440604600031007003,440604679361002000,113.139246,23.015031,113.114211,23.004786,6,小型客车,汽油,国6,总时间误差满足


In [42]:
# 单次出行重构轨迹统计指标
print("平均出行时长(min): ", round(np.mean(test_data['travel_time_s'])/60, 2), round(np.max(test_data['travel_time_s'])/60, 2))
print("平均出行距离(km): ", round(np.mean(test_data['distance_km']), 2), round(np.max(test_data['distance_km']), 2))
print("平均出行速度(km/h): ", round(np.mean(test_data['speed_kmh']), 2), round(np.max(test_data['speed_kmh']), 2))

# 统计出行距离分布
bins = np.arange(0, 71, 5)
labels = np.arange(5, 71, 5)
test_data['distance_group'] = pd.cut(test_data['distance_km'], bins=bins, labels=labels)
distance_grouped = test_data.groupby('distance_group').agg({'CLTMBH': 'count'})
distance_grouped = distance_grouped.reset_index(drop=False)
distance_grouped = distance_grouped.rename(columns={'distance_group': 'distance_km', 'CLTMBH': 'paths'})

# 统计出行时间分布
test_data['travel_time_min'] = round(test_data['travel_time_s'] / 60)
bins = np.arange(0, 181, 10)
labels = np.arange(10, 181, 10)
test_data['travel_group'] = pd.cut(test_data['travel_time_min'], bins=bins, labels=labels)
travel_grouped = test_data.groupby('travel_group').agg({'CLTMBH': 'count'})
travel_grouped = travel_grouped.reset_index(drop=False)
travel_grouped = travel_grouped.rename(columns={'travel_group': 'travel_time_min', 'CLTMBH': 'paths'})

# 保存数据
# distance_grouped.to_excel('重构轨迹的行驶距离分布.xlsx', index=False)
# travel_grouped.to_excel('重构轨迹的行程时间分布.xlsx', index=False)

平均出行时长(min):  12.98 172.85
平均出行距离(km):  7.0 67.24
平均出行速度(km/h):  33.48 60.0


In [3]:
# 统计每一个卡口对应的路网节点经纬度

# 1. 使用 melt() 将 start_bayonet 和 stop_bayonet 变成长格式，提高处理效率
df_melted = test_data.melt(id_vars=['start_node_lon', 'start_node_lat', 'stop_node_lon', 'stop_node_lat'],
                           value_vars=['start_bayonet', 'stop_bayonet'],
                           var_name='bayonet_type', value_name='bayonet')

# 2. 根据 `bayonet_type` 选择正确的经纬度列
df_melted['node_lon'] = df_melted.apply(lambda row: row['start_node_lon'] if row['bayonet_type'] == 'start_bayonet' else row['stop_node_lon'], axis=1)
df_melted['node_lat'] = df_melted.apply(lambda row: row['start_node_lat'] if row['bayonet_type'] == 'start_bayonet' else row['stop_node_lat'], axis=1)

# 3. 选择最终列，并去重
bayonet_df = df_melted[['bayonet', 'node_lon', 'node_lat']].drop_duplicates()
bayonet_coords = bayonet_df.reset_index(drop=True)

In [27]:
def categorize_time_period(time):
    """标记时间段（高峰期、平峰期、低峰期）"""
    if time.hour in range(7, 9) or time.hour in range(17, 19):  # 高峰期
        return 'high_peak'
    elif time.hour in range(22, 24) or time.hour in range(0, 7):  # 低峰期
        return 'low_peak'
    else:
        return 'off_peak'  # 平峰期

def peak_od_flow(peakdf):
    """计算不同时间段的od流量占比"""
    peakdf = peakdf.reset_index(drop=True)
    peakdf['perc'] = peakdf['flow'] / sum(peakdf['flow'])
    peakdf["perc_cumsum"] = peakdf["perc"].cumsum(axis=0)
    # 仅保留出行次数占比前5%的OD
    # peakdf = peakdf[peakdf['perc_cumsum'] <= 0.05].reset_index(drop=True)
    # 仅保留出行次数前100的OD
    peakdf = peakdf.iloc[0:100]
    return peakdf

def peak_bayonet_flow(peakdf_od_statics, bayonet_coords, period):
    """统计不同时间段的卡口设备流量占比"""
    point_melted = peakdf_od_statics.melt(id_vars=['flow'], 
                                          value_vars=['start_bayonet', 'stop_bayonet'], 
                                          var_name='bayonet_type', value_name='bayonet') # 将 start_bayonet 和 stop_bayonet 归为同一列
    pointdf = point_melted.groupby('bayonet')['flow'].sum().reset_index()
    # 匹配卡口节点的经纬度
    pointdf = pd.merge(pointdf, bayonet_coords, on='bayonet', how='left')
    pointdf['time_period'] = period
    return pointdf

def od_flow(df, bayonet_coords):
    # 统计每一出行OD对的车流量
    df['time_period'] = df['start_time'].apply(categorize_time_period)
    od_statics = df.groupby(['time_period', 'start_bayonet', 'stop_bayonet']).agg({'CLTMBH': 'count'})
    od_statics = od_statics.reset_index(drop=False)
    od_statics = od_statics.rename(columns={'CLTMBH': 'flow'})
    od_statics = od_statics.sort_values(by=['time_period', 'flow'], ascending=False).reset_index(drop=True)  # 按照车流量由高到低排序

    # 根据高峰、低峰、平峰统计OD对的流量占比
    high_od_statics = peak_od_flow(od_statics[od_statics['time_period'] == 'high_peak'])
    low_od_statics = peak_od_flow(od_statics[od_statics['time_period'] == 'low_peak'])
    off_od_statics = peak_od_flow(od_statics[od_statics['time_period'] == 'off_peak'])
    # 合并统计结果
    od_statics = pd.concat([high_od_statics, low_od_statics], ignore_index=True)
    od_statics = pd.concat([od_statics, off_od_statics], ignore_index=True)
    
    # 匹配OD的经纬度信息
    coords = df[['start_bayonet', 'stop_bayonet', 'start_node_lon', 'start_node_lat', 'stop_node_lon', 'stop_node_lat']]
    coords = coords.drop_duplicates(subset=['start_bayonet', 'stop_bayonet'], keep='first')
    od_statics = pd.merge(od_statics, coords, on=['start_bayonet', 'stop_bayonet'], how='left')
    
    # 构建 GeoDataFrame 点转线
    geomList = []
    for i in range(0, len(od_statics)):
        start_point = (od_statics.at[i, 'start_node_lon'], od_statics.at[i, 'start_node_lat'])
        stop_point = (od_statics.at[i, 'stop_node_lon'], od_statics.at[i, 'stop_node_lat'])
        line = LineString([start_point, stop_point])
        geomList.append(line)
    od_statics_geo = gpd.GeoDataFrame(od_statics, crs='EPSG:4326', geometry=geomList)
    
    # 统计每个OD点的双向流量
    high_point_flow = peak_bayonet_flow(high_od_statics, bayonet_coords, 'high_peak')
    low_point_flow = peak_bayonet_flow(low_od_statics, bayonet_coords, 'low_peak')
    off_point_flow = peak_bayonet_flow(off_od_statics, bayonet_coords, 'off_peak')
    # 合并统计结果
    point_flow = pd.concat([high_point_flow, low_point_flow], ignore_index=True)
    point_flow = pd.concat([point_flow, off_point_flow], ignore_index=True)
    
    # 构建 GeoDataFrame
    geomPoint = [Point(xy) for xy in zip(point_flow['node_lon'], point_flow['node_lat'])]
    point_flow_geo = gpd.GeoDataFrame(point_flow, crs='EPSG:4326', geometry=geomPoint)
    
    return od_statics_geo, point_flow_geo

bus_od_statics, bus_bayonet_statics = od_flow(bus_test_data, bayonet_coords)
lighttruck_od_statics, lighttruck_bayonet_statics = od_flow(lighttruck_test_data, bayonet_coords)
heavytruck_od_statics, heavytruck_bayonet_statics = od_flow(heavytruck_test_data, bayonet_coords)

print('客车出行高频od:', len(bus_od_statics))
print('微轻型货车出行高频od:', len(lighttruck_od_statics))
print('中重型货车出行高频od:', len(heavytruck_od_statics))

bus_bayonet_statics.head(3)

客车出行高频od: 300
微轻型货车出行高频od: 300
中重型货车出行高频od: 300


,bayonet,flow,node_lon,node_lat,time_period,geometry
0,440604305050002400,285,113.091540,23.044349,high_peak,POINT (113.09154 23.04435)
1,440604305050002401,168,113.091053,23.044593,high_peak,POINT (113.09105 23.04459)
2,440604305050003500,139,113.070463,23.047882,high_peak,POINT (113.07046 23.04788)


In [28]:
# 保存为 Shapefile 文件
save_file = r'E:\硕士毕业论文update250120\案例分析\ArcGIS\车辆出行空间分布特征\od分布'

# OD流量
bus_od_statics[bus_od_statics['time_period'] == 'high_peak'].to_file(save_file + '/客车/' + 'bus_highpeak_od.shp', driver='ESRI Shapefile', encoding='utf-8')
bus_od_statics[bus_od_statics['time_period'] == 'low_peak'].to_file(save_file + '/客车/' + 'bus_lowpeak_od.shp', driver='ESRI Shapefile', encoding='utf-8')
bus_od_statics[bus_od_statics['time_period'] == 'off_peak'].to_file(save_file + '/客车/' + 'bus_offpeak_od.shp', driver='ESRI Shapefile', encoding='utf-8')

lighttruck_od_statics[lighttruck_od_statics['time_period'] == 'high_peak'].to_file(save_file + '/微轻型货车/' + 'lighttruck_highpeak_od.shp', driver='ESRI Shapefile', encoding='utf-8')
lighttruck_od_statics[lighttruck_od_statics['time_period'] == 'low_peak'].to_file(save_file + '/微轻型货车/' + 'lighttruck_lowpeak_od.shp', driver='ESRI Shapefile', encoding='utf-8')
lighttruck_od_statics[lighttruck_od_statics['time_period'] == 'off_peak'].to_file(save_file + '/微轻型货车/' + 'lighttruck_offpeak_od.shp', driver='ESRI Shapefile', encoding='utf-8')

heavytruck_od_statics[heavytruck_od_statics['time_period'] == 'high_peak'].to_file(save_file + '/中重型货车/' + 'heavytruck_highpeak_od.shp', driver='ESRI Shapefile', encoding='utf-8')
heavytruck_od_statics[heavytruck_od_statics['time_period'] == 'low_peak'].to_file(save_file + '/中重型货车/' + 'heavytruck_lowpeak_od.shp', driver='ESRI Shapefile', encoding='utf-8')
heavytruck_od_statics[heavytruck_od_statics['time_period'] == 'off_peak'].to_file(save_file + '/中重型货车/' + 'heavytruck_offpeak_od.shp', driver='ESRI Shapefile', encoding='utf-8')

# 卡口流量
bus_bayonet_statics[bus_bayonet_statics['time_period'] == 'high_peak'].to_file(save_file + '/客车/' + 'bus_highpeak_bayonet.shp', driver='ESRI Shapefile', encoding='utf-8')
bus_bayonet_statics[bus_bayonet_statics['time_period'] == 'low_peak'].to_file(save_file + '/客车/' + 'bus_lowpeak_bayonet.shp', driver='ESRI Shapefile', encoding='utf-8')
bus_bayonet_statics[bus_bayonet_statics['time_period'] == 'off_peak'].to_file(save_file + '/客车/' + 'bus_offpeak_bayonet.shp', driver='ESRI Shapefile', encoding='utf-8')

lighttruck_bayonet_statics[lighttruck_bayonet_statics['time_period'] == 'high_peak'].to_file(save_file + '/微轻型货车/' + 'lighttruck_highpeak_bayonet.shp', driver='ESRI Shapefile', encoding='utf-8')
lighttruck_bayonet_statics[lighttruck_bayonet_statics['time_period'] == 'low_peak'].to_file(save_file + '/微轻型货车/' + 'lighttruck_lowpeak_bayonet.shp', driver='ESRI Shapefile', encoding='utf-8')
lighttruck_bayonet_statics[lighttruck_bayonet_statics['time_period'] == 'off_peak'].to_file(save_file + '/微轻型货车/' + 'lighttruck_offpeak_bayonet.shp', driver='ESRI Shapefile', encoding='utf-8')

heavytruck_bayonet_statics[heavytruck_bayonet_statics['time_period'] == 'high_peak'].to_file(save_file + '/中重型货车/' + 'heavytruck_highpeak_bayonet.shp', driver='ESRI Shapefile', encoding='utf-8')
heavytruck_bayonet_statics[heavytruck_bayonet_statics['time_period'] == 'low_peak'].to_file(save_file + '/中重型货车/' + 'heavytruck_lowpeak_bayonet.shp', driver='ESRI Shapefile', encoding='utf-8')
heavytruck_bayonet_statics[heavytruck_bayonet_statics['time_period'] == 'off_peak'].to_file(save_file + '/中重型货车/' + 'heavytruck_offpeak_bayonet.shp', driver='ESRI Shapefile', encoding='utf-8')

# 2. 热点区域统计（以路网节点->格网为单位）

In [3]:
# 读取每一路网节点速度估计结果 2021年12月1日 4:00/7:00/12:00 佛山市中心城区燃油客车、货车速度估计结果 对应每一路网节点

folder_path2 = r'E:\硕士毕业论文update250120\案例分析\数据\速度估计结果'
target_data = pd.read_csv(folder_path2 + '/' + 'test_data_speed.csv', encoding='gbk')
target_data['time'] = pd.to_datetime(target_data['time'])

# 根据客车、微型&轻型货车、中重型货车分类

# 匹配车辆信息
vehicle_info = pd.read_csv(r'C:\Users\chenii\Code(Python)\硕士毕业论文\应用案例分析\车辆路径重构\2.重构车辆路径\卡口数据预处理\中心城区研究车辆信息表.csv', encoding='gbk')
vehicle_info = vehicle_info[['CLTMBH', 'pfbz', 'veh_type', 'fuel_type']]
vehicle_info.rename(columns={'CLTMBH': 'device_id'}, inplace=True)
target_data = pd.merge(target_data, vehicle_info, on='device_id', how='left')

# 划分数据集
bus_target_data = target_data[target_data['veh_type'].str.contains('客车')]
truck_target_data = target_data[target_data['veh_type'].str.contains('货车')]
lighttruck_target_data = truck_target_data[truck_target_data['veh_type'].isin(['微型货车', '轻型货车'])]
heavytruck_target_data = truck_target_data[truck_target_data['veh_type'].isin(['中型货车', '重型货车'])]

print('2021年12月1日4:00/7:00/12:00 每一路网节点的速度估计结果:', len(target_data))
print('客车占比: {:.2%}'.format(len(bus_target_data) / len(target_data)),
      '微轻型货车占比: {:.2%}'.format(len(lighttruck_target_data) / len(target_data)),
      '中重型货车占比: {:.2%}'.format(len(heavytruck_target_data) / len(target_data)))

bus_target_data.head(3)

2021年12月1日4:00/7:00/12:00 每一路网节点的速度估计结果: 3226982
客车占比: 98.32% 微轻型货车占比: 0.93% 中重型货车占比: 0.74%


,device_id,travel_id,date,time,bayonetname,posname,longitude,latitude,distance,c_distance,edge_length,edge_index,time_period,road_type,in_urban_area,predict_speed,pfbz,veh_type,fuel_type
0,粤E000211ee60ae0d3d294befa412bfca49,15,2021-12-01,2025-01-31 07:08:19,4.406046e+17,LINK-196858391-672869084-672869096,113.138836,22.998446,0.00,0.00,15.80,24199,high_peak,主干道,True,0.972734,国5,小型客车,汽油
1,粤E000211ee60ae0d3d294befa412bfca49,15,2021-12-01,2025-01-31 07:08:22,NaN,LINK-672869095-672869096-672869097-672869098,113.139072,22.998533,25.96,25.96,69.55,24254,high_peak,主干道,True,17.994352,国5,小型客车,汽油
2,粤E000211ee60ae0d3d294befa412bfca49,15,2021-12-01,2025-01-31 07:08:53,NaN,1-672869098,113.139080,23.001244,300.25,326.21,21.60,26725,high_peak,主干道,True,30.954689,国5,小型客车,汽油


In [4]:
# 提取每一路网节点对应的经纬度
pos_coords = target_data[['posname', 'longitude', 'latitude']].drop_duplicates(subset='posname', keep='first')
pos_coords = pos_coords.reset_index(drop=True)
print('经过的路网节点总数:', len(pos_coords))

# 读取格网数据
gewang_path = r'E:\硕士毕业论文update250120\案例分析\ArcGIS\佛山市中心区网格数据'
grid = gpd.read_file(gewang_path + '/' + '中心城区网格500m_WGS_new.shp', crs='EPSG:4326')  # 读取格网
print('500*500m格网的个数:', len(grid))

经过的路网节点总数: 8257
500*500m格网的个数: 607


In [14]:
def peak_pos_info(peakdf, pos_coords):
    """计算不同时间段、每一路网节点的车流量、平均运行速度"""
    pos_statics = peakdf.groupby(['posname']).agg({'device_id': 'count', 'predict_speed': 'mean'})
    pos_statics = pos_statics.reset_index(drop=False)
    pos_statics = pos_statics.rename(columns={'device_id': 'flow', 'predict_speed': 'meanspeed_kmh'})
    # 匹配路网节点的经纬度
    pos_statics = pd.merge(pos_statics, pos_coords, on='posname', how='left')
    # 构建 GeoDataFrame
    geomPoint = [Point(xy) for xy in zip(pos_statics['longitude'], pos_statics['latitude'])]
    pos_statics_geo = gpd.GeoDataFrame(pos_statics, crs='EPSG:4326', geometry=geomPoint) # WGS_984 EPSG:4326
    return pos_statics_geo


def grid_statics(road_nodes, grid, folder, vehicle_type, period):
    """统计每一个格网内的总流量和平均速度"""
    # 进行空间连接，将点数据关联到对应的格网
    joined = gpd.sjoin(road_nodes, grid, how="right", predicate="within")
    # 按照格网 uid 进行聚合
    grid_stats = joined.groupby("uid").agg(
        total_flow=("flow", "sum"),  # 计算流量总和
        mean_speed=("meanspeed_kmh", "mean")  # 计算平均速度
    ).reset_index()
    # 用前后非零值填充
    grid_stats["total_flow"] = grid_stats["total_flow"].ffill().bfill()
    grid_stats["mean_speed"] = grid_stats["mean_speed"].ffill().bfill()
    # 将结果合并回格网数据
    grid_result = grid.merge(grid_stats, on="uid", how="left")
    # 处理无匹配值的格网（NaN 替换为 0）
    grid_result.fillna({"total_flow": 0, "mean_speed": 30}, inplace=True)
    # 计算流量占比
    grid_result['flow_perc'] = grid_result['total_flow'] / sum(grid_result['total_flow'])
    # 保存 shapefile
    filename = vehicle_type + '_' + period + '_500m.shp'
    grid_result.to_file(folder + '/' + filename, driver='ESRI Shapefile', encoding='utf-8')
    return grid_result


def pos_info(df, pos_coords, grid, folder, vehicle_type):
    
    # 计算不同时间段、每一路网节点的车流量、平均运行速度
    high_pos_statics = peak_pos_info(df[df['time_period'] == 'high_peak'], pos_coords)
    low_pos_statics = peak_pos_info(df[df['time_period'] == 'low_peak'], pos_coords)
    off_pos_statics = peak_pos_info(df[df['time_period'] == 'off_peak'], pos_coords)
    
    # 统计不同时段、每一个格网的总流量和平均速度
    high_grid_statics = grid_statics(high_pos_statics, grid, folder, vehicle_type, 'highpeak')
    low_grid_statics = grid_statics(low_pos_statics, grid, folder, vehicle_type, 'lowpeak')
    off_grid_statics = grid_statics(off_pos_statics, grid, folder, vehicle_type, 'offpeak')
    
    return high_grid_statics, low_grid_statics, off_grid_statics

In [7]:
# 运行程序
bus_folder = r'E:\硕士毕业论文update250120\案例分析\ArcGIS\车辆出行空间分布特征\热点区域\客车'
bus_high_grid_statics, bus_low_grid_statics, bus_off_grid_statics = pos_info(bus_target_data, pos_coords, grid, bus_folder, 'bus')

lighttruck_folder = r'E:\硕士毕业论文update250120\案例分析\ArcGIS\车辆出行空间分布特征\热点区域\微轻型货车'
lighttruck_high_grid_statics, lighttruck_low_grid_statics, lighttruck_off_grid_statics = pos_info(lighttruck_target_data, pos_coords, grid, lighttruck_folder, 'lighttruck')

heavytruck_folder = r'E:\硕士毕业论文update250120\案例分析\ArcGIS\车辆出行空间分布特征\热点区域\中重型货车'
heavytruck_high_grid_statics, heavytruck_low_grid_statics, heavytruck_off_grid_statics = pos_info(heavytruck_target_data, pos_coords, grid, heavytruck_folder, 'heavytruck')

bus_high_grid_statics.head(3)

,uid,geometry,total_flow,mean_speed,flow_perc
0,1,"POLYGON ((113.05834 23.00866, 113.05346 23.008...",913.0,25.338265,0.000419
1,2,"POLYGON ((113.15594 22.95476, 113.15106 22.954...",0.0,25.338265,0.000000
2,3,"POLYGON ((113.04370 23.01315, 113.03882 23.013...",2802.0,25.130109,0.001284


# 3. 路网速度分布

In [15]:
road_folder = r'E:\硕士毕业论文update250120\案例分析\ArcGIS\车辆出行空间分布特征\路网速度'
allveh_high_grid_statics, allveh_low_grid_statics, allveh_off_grid_statics = pos_info(target_data, pos_coords, grid, road_folder, 'allveh')

allveh_high_grid_statics.head(3)

,uid,geometry,total_flow,mean_speed,flow_perc
0,1,"POLYGON ((113.05834 23.00866, 113.05346 23.008...",930.0,25.271290,0.000423
1,2,"POLYGON ((113.15594 22.95476, 113.15106 22.954...",0.0,25.271290,0.000000
2,3,"POLYGON ((113.04370 23.01315, 113.03882 23.013...",2828.0,25.101008,0.001287
